In [7]:
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
import random
import math
import torch
from torch import nn
from torch import optim
from tensorboardX import SummaryWriter
from datetime import datetime
import time
import load
from models import DynamicTopologyModel
import load_topology_dataset
from load_topology_dataset import MAX_CELL_COUNT, MIN_CELL_COUNT, IMAGE_SIZE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
EXPERIMENT=16
writer = SummaryWriter('runs/experiment_{}/'.format(EXPERIMENT))

In [8]:
def save_parameters(writer, model, batch_idx):
    for k, v in model.state_dict().items():
        shape = v.shape
        # Don't do this for single weights or biases
        if np.any(np.array(shape) > 1):
            mean = torch.mean(v)
            std_dev = torch.std(v)
            maximum = torch.max(v)
            minimum = torch.min(v)
            writer.add_scalars("{}_{} ".format(k, shape), {"mean": mean,
                                                                    "std_dev": std_dev,
                                                                    "max": maximum,
                                                                    "min": minimum}, batch_idx)
            writer.add_histogram("{}_{}".format(k, shape), v, batch_idx)
            
        else:
            writer.add_scalar("{}_{}".format(k, shape), v.data, batch_idx)

In [9]:
def test_parameters_update(last_parameters_np ,parameters_np):
    for last_param, param in zip(last_parameters_np, parameters_np):
        diff = param - last_param
        #print(last_parameters.max())
        print(diff.max())
        #print(not np.all(diff))


In [10]:
def print_example(reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
                  reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
                  neighbourhood_cell_present_target):
    print("reference_cell_input") 
    print(reference_cell_input)
    print("reference_cell_present_input") 
    print(reference_cell_present_input)
    print("neighbourhood_cell_rel_input") 
    print(neighbourhood_cell_rel_input)
    print("neighbourhood_cell_load_input") 
    print(neighbourhood_cell_load_input)
    print("neighbourhood_cell_present_input") 
    print(neighbourhood_cell_present_input)
    print("reference_cell_target") 
    print(reference_cell_target)
    print("reference_cell_present_target") 
    print(reference_cell_present_target)
    print("neighbourhood_cell_rel_target") 
    print(neighbourhood_cell_rel_target)
    print("neighbourhood_cell_present_target") 
    print(neighbourhood_cell_present_target)

In [ ]:

# BATCH SIZE STILL NOT WORKING!!!

BATCH_SIZE =16 
INPUT_SEQ_LEN = 8 
TARGET_SEQ_LEN = 8
LEARNING_RATE = 1e-3
SAVE_VARIABLE_EVERY = 100
DATASET_WORKERS = 2

# Define Data.
cell_load_dataset = load_topology_dataset.LoadCellDataset(initial_cell_counts=(2, 5), initial_load_counts=(2,5),
                                    input_seq_len=INPUT_SEQ_LEN, target_seq_len=TARGET_SEQ_LEN, network_mutate_prob=[0.5, 0.5
                                                                                                                    ])
dataloader = DataLoader(cell_load_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=DATASET_WORKERS)

# Define Model
dynamic_topology_model = DynamicTopologyModel(neighbourhood_hidden_size=32, neighbourhood_cell_count=6,
                     neighbourhood_output_size=16, lstm_hidden_size=32, lstm_layers=2, teacher_forcing_probability=0.5, device=device).to(device)

parameters = dynamic_topology_model.parameters

# Define optimizer
optimizer = optim.Adam(parameters
                       , lr=LEARNING_RATE)
criterion = nn.MSELoss()
last_parameters_np = list(np.copy(param.cpu().detach().numpy()) for param in parameters)

# Train
start = datetime.now()
for batch_idx, data in enumerate(dataloader):
    optimizer.zero_grad()
    reference_cell_input =  data[0].to(device)
    reference_cell_present_input = data[1].to(device)
    neighbourhood_cell_rel_input = data[2].to(device)
    neighbourhood_cell_load_input = data[3].to(device)
    neighbourhood_cell_present_input = data[4].to(device)
    reference_cell_target = data[5].to(device)
    reference_cell_present_target = data[6].to(device)
    neighbourhood_cell_rel_target = data[7].to(device)
    neighbourhood_cell_present_target = data[8].to(device)
    
    #print_example(reference_cell_input, reference_cell_present_input,
    #            neighbourhood_cell_rel_input, neighbourhood_cell_load_input, neighbourhood_cell_present_input,
    #              reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target,
    #              neighbourhood_cell_present_target)    
    
    decoder_output_seq = dynamic_topology_model.forward(INPUT_SEQ_LEN, TARGET_SEQ_LEN, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
    
    loss = criterion(decoder_output_seq, reference_cell_target)
    loss.backward()
    optimizer.step()
    if batch_idx == 0:
        parameters = dynamic_topology_model.parameters
        parameters_np = list(param.cpu().detach().numpy() for param in parameters)
        test_parameters_update(last_parameters_np, parameters_np)
    if batch_idx % SAVE_VARIABLE_EVERY == 0:
        end = datetime.now()
        print("{} Examples/sec".format((SAVE_VARIABLE_EVERY* BATCH_SIZE) / ((end - start).total_seconds()) ))
        start = datetime.now()
        print("loss: {}".format(loss))
        #print("output: {}".format(decoder_output_seq))
        #print("target: {}".format(reference_cell_target))
        inputs = (INPUT_SEQ_LEN, TARGET_SEQ_LEN, reference_cell_input, reference_cell_present_input,
                neighbourhood_cell_rel_input, neighbourhood_cell_load_input, reference_cell_target, reference_cell_present_target, neighbourhood_cell_rel_target)
        #writer.add_graph(dynamic_topology_model, inputs, True)
        writer.add_scalar("loss", loss, batch_idx)
        save_parameters(writer, dynamic_topology_model, batch_idx)

0.0009601712
0.0009871721
0.000992924
0.0009997487
0.0008211434
-0.0009979159
0.000999961
0.0009999648
0.0009999946
0.0
0.0009999722
0.0009999797
0.0009999573
0.0
0.0009999946
0.0009999946
0.0
0.0
0.0009998679
0.001000002
0.001000002
0.001000002
0.001000002
0.0009999946
0.001000002
0.001000002
0.001000002
0.0009999946
0.001000002
0.001000002
0.001000002
0.0009999983
12187.968951149096 Examples/sec
loss: 10.030384063720703
496.3893876912806 Examples/sec
loss: 2.006837844848633
500.3680832712564 Examples/sec
loss: 0.6647555828094482
536.0943981821039 Examples/sec
loss: 0.3385075628757477
534.24796585087 Examples/sec
loss: 0.21052958071231842
534.3371747723389 Examples/sec
loss: 0.32429754734039307
532.7840329953152 Examples/sec
loss: 0.02895534224808216
534.2918529174673 Examples/sec
loss: 0.04800834134221077
533.8698725552514 Examples/sec
loss: 0.003719710512086749
534.7339798712762 Examples/sec
loss: 0.02778007462620735
535.8294053130834 Examples/sec
loss: 0.0270879827439785
528.020043

In [ ]:
len(reference_cell_input.shape)